# Data science with xarray

Hello and welcome to your intro to `xarray` for data science as part of NCI's data science course.

This notebook is designed to cover the fundamentals of `xarray` highlighting some similarities with tools you have already been introduced to such as `NumPy` and `CuPy` and some of the advantages of using `xarray`.

The xarray project has extensive and well explained documentation at the xarray [homepage](https://docs.xarray.dev/en/stable/index.html). 


In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the folder
# which was created in your username directory under /scratch/vp91

#TODO 
os.chdir(os.path.expandvars("/scratch/vp91/$USER/Data-Analytics/"))


## Intro

First of all what is `xarray`? `xarray` is a project that came out of climate and geophysics research, recognising the need for rapid, scalable and easily manipulated N dimensional array data **with labels and metadata**. 

But wait, doesn't `NumPy` provide an N Dimensional array? Yes it does, but `xarray` provides the ability to use labelled and metadata rich arrays, providing to quote the manual a " more intuitive, more concise, and less error-prone developer experience".

This is done by providing a label based API that simplifies a lot of the manual book keeping of working with `numpy` arrays directly.

No longer will you forget what experiment this data came from, what that tensor dimension was, or what array position corresponded to with time point! Even better these, augmented N dimensional arrays can be combined to make massive datasets, enabling analysis of huge volumes of data. It's all there in  the one datastructure that can be easily stored and shared, as is done on a massive scale using `xarray` in the climate modelling, geophysics, and astrophysics communities to name just a few. 


Alright, lets jump in! 

## Crash course

In [ ]:
# lets import xarray and numpy
import xarray as xr
import numpy as np


The `DataArray` is the `xarray` equivalent of the `numpy` `ndarray` and will be the first focus of our intro. Lets make a 2 x 3 DataArray using a numpy array, so we can get the hang of how to work with it. 

In [ ]:
inp = np.arange(6).reshape(2,3)
data = xr.DataArray(inp, dims=("x", "y"))
data

We can see that our data array looks a lot like a numpy array, but with two labelled dimensions `x` and `y`. We can access our values directly using the `values` attribute as shown below. 

In [ ]:
data.values

We can also access our dimensions using the `dims` attribute as shown below

In [ ]:
data.dims

The eagle eyed amongst you may have  spotted the `coords` attribute. This is used to associate a value along a particular axis with another value, that could for example correspond to the time or location it was measured. The possibilities are only limited by your imagination

We set our `coords` using a dictionary corresponding to the `coords` along a particular axis.

In [ ]:
coords = {"x":[10, 20], "y":[0.1, 0.2, 0.3]}
data = data.assign_coords(coords)
data

**Notice the detail above**, we had to assign our `DataArray` to a new object (in this case we overwrote `data`) for our change to persist. This pattern will be familiar to those who use `pandas`.


### Challenge
Make a different set of coordinates for `data` and assign it to a new variable.


In [ ]:
# make a different set of coordinates for data and assign it to a new variable.


<details><summary><b>Solution</b></summary>
   <pre>
    <br>coords = {"x":[2.0, 2.1], "y":[300, 400, 500]}
data2 = data.assign_coords(coords)
   </pre>
</details>

## Indexing

Okay cool, how can we access the data in our array? We can use four kinds of slicing. 

* Numpy like using integer locations
* using `loc` like in pandas
* using an integer select (`isel`), combining a dimension name and integer label
* using a selection (`sel`) based on coordinate combining a dimension name and coordinate value

The following selections all give the same set of values, those at `x=10`:

**Numpy like using array indexing:**

In [ ]:
data[0, :]

**Pandas like using `loc`**

In [ ]:
data.loc[10]

**An xarray integer selection**

In [ ]:
data.isel(x=0,y=2)

**An xarray selection**

In [ ]:
data.sel(x=10,y=0.3)

When dealing with a complicated multidimensional dataset, I and the creators of `xarray` would argue that the last two are simple and powerful.

Note that we can also use `pandas` style `DataArray.dim` indexing to access a given dimension eg:


In [ ]:
data.x

aggregation operations can also use dimension labels eg:

In [ ]:
data.mean(dim="x")

### Challenge

select the slice of data at `y=0.2` and compute its mean.


In [ ]:
# select the slice of data at `y=0.2` and compute its mean.


<details><summary><b>Solution</b></summary>
   <pre>
    <br> data.sel(y=0.2).mean()
   </pre>
</details>

## Metadata

When dealing with complex data another advantage of `xarray` is metadata. Using the `.attrs`  `dict` attribute of `DataArray` we can associate any arbitrary field with our array data. For example lets label our current dataset:


In [ ]:
data.attrs["long_name"] = "random array"
data.attrs["units"] = "rad/second"
data.attrs["description"] = "A random variable created as an example."
data.attrs["arbitrary"] = "arbitrary"
data

We can also associate our dimensions with metadata. For example lets add some metadata with our `x` dimension. 

Note here that we use pandas style `data.dim` accessing as demonstrated above.

In [ ]:
data.x.attrs["units"] = "x units"
data.x.attrs["name"] = "name of x dimension"
data.x

Of course, just like a numpy array we can do broadcasted operations accross the whole array

In [ ]:
data += 100
data

And use many of the standard numpy style shortcuts such as `.T` 

In [ ]:
data.T

## Broadcasting, operations and labelled dimensions

A fantastic feature of xarray is that operations are dimension specific. Let me show you what I mean.

We are going to make three arrays for demonstration purposes. Array `x` and `y` have different `dims` while `x` and `z` share the `x` dimension.

In [ ]:
# has x as dimension
x = xr.DataArray(np.arange(10), dims="x", coords= {"x": np.arange(10)/10})

# has y as dimension
y = xr.DataArray(np.arange(3), dims="y", coords = {"y": np.arange(3)/10})

# has x dimension
z = xr.DataArray(np.arange(3), dims="x", coords = {"x": np.arange(3)/10})

So what happens when I do an operation on arrays with non-matching dimensions?


In [ ]:
print(x)
print(y)
print(z)


In [ ]:
# result broadcast
result = x*y
result

In [ ]:
y*x

Thats interesting! We actually get an **outer product** (tensor product for more than two matching dimensions) formed by broadcasting accross the non-matching dimensions.  

When dimensions and coordinates match, we can easily broadcast accross those instead.

In [ ]:
# result broadcast on matching dimension and coordinates
result = x*z
result


The net result of all of this is that we don't need to manually keep track of dimension ordering, indexing and alignment of our coordinates as you would have to do in numpy and don't have to pad arrays with dummy data.

As an example of this, lets multiply our array by its transpose, noting that it is not a square matrix.

In [ ]:
ten_by_three = x*y
ten_by_three

In [ ]:
ten_by_three_transpose = ten_by_three.T
ten_by_three_transpose

In [ ]:
result = ten_by_three*ten_by_three.T
result

here, xarray took care of aligning our dimensions based on **dimension labels** even though the order of dimensions was reversed in our transpose.

This is not possible in numpy as show below

### Challenge

Uncomment the below and see what happens.

In [ ]:
# you can uncomment this and find out

# a = np.outer(np.arange(10), np.arange(3))
# a*a.T

## Saving data and serialising

A lot of the functionality in `xarray` is based on the netcdf binary format, and as such the reccomended format for saving a `DataArray` is as a netcdf file. 

We need to import either `scipy` or `netcdf4-python` to provide the interface with the format. 

In [ ]:
import scipy
result.to_netcdf("array.nc")

## Worked example with a 4D function

We are going to go through a more in depth example of how `xarray's` functionality is such a powerful tool.

We are going to use an example of a 3D mesh that various over time, resulting in a 4D function with dimensions (x,y,z,t). We are going to use our imagination and pretend that these are **waves propagating  noisily in a 3D box**

 First we construct our domain:

In [ ]:
# construct our domain
spatial_density = 40
time_density = 1001
spatial_domain = (-5,5)
time_domain = (0,10)

Then our grids

In [ ]:
# turn into grid points
x = np.linspace(spatial_domain[0], spatial_domain[1], spatial_density)
y = np.linspace(spatial_domain[0], spatial_domain[1], spatial_density)
z = np.linspace(spatial_domain[0], spatial_domain[1], spatial_density)
time = np.linspace(time_domain[0], time_domain[1], time_density)

Lets convert our grids to `DataArrays` early in our computation

In [ ]:
x = xr.DataArray(x, coords={"x":x})
y = xr.DataArray(y, coords={"y":y})
z = xr.DataArray(z, coords={"z":z})
time = xr.DataArray(time, coords={"time":time})

now lets define our function and use some xarray broadcasting to make it fast and automagically work with coordinates using `xr.apply_ufunc()` 

In [ ]:
def time_dependent_grid(x, y, z, t):
    func = lambda x_, y_, z_, t_: np.cos(1*x_ + 2*y_ + 3*z_ + np.sin(x_*y_*t_))
    return xr.apply_ufunc(func, x, y, z, t)
    


Making use of our auto-broadcasting we now call our function on our grid, and get back values on a `40x40x40x1001` grid with **over 64M points**.

In [ ]:
function_on_grid = time_dependent_grid(x,y,z,time)
function_on_grid

Lets not forget to add metadata so that our object makes sense for someone else



In [ ]:
# add metadata to overall array
function_on_grid.attrs["name"] = "wave intensity"
function_on_grid.attrs["units"] = " wavenumbers"
function_on_grid.attrs["description"] = "A wave propagation model used as an xarray example"

#add metadata to dimensions
function_on_grid.x.attrs["name"] = "X dimension"
function_on_grid.x.attrs["units"] = "meters"

function_on_grid.y.attrs["name"] = "Y dimension"
function_on_grid.y.attrs["units"] = "meters"

function_on_grid.z.attrs["name"] = "Z dimension"
function_on_grid.z.attrs["units"] = "meters"


function_on_grid.time.attrs["name"] = "time dimension"
function_on_grid.time.attrs["units"] = "seconds"


Now we can save it for someone else to use later 

In [ ]:
function_on_grid.to_netcdf("wavemodel.nc")

# Plotting

Now that we have created our wave model, lets have a look at it at some time and space slices just for a bit of fun. Remember that to plot a 4D function in 3D we need to fix **2** coordinates. Here  we use the powerful xarray selection syntax to pull out our function values at `t=0` and `x=-5.0` and make plots using their super easy interface.

In [ ]:
func2dt0 = function_on_grid.sel(time=0.0, x=-5.0)
func2dt0.plot.surface()


We can also easily make a contour plot

In [ ]:
func2dt0.plot.contour()

Okay what happens when we move forward one time unit at a time?

In [ ]:
func2d_t1 = function_on_grid.sel(time=1.0, x=-5.0)
func2d_t1.plot.surface()

In [ ]:
func2d_t1.plot.contour()

In [ ]:
func2d_t2 = function_on_grid.sel(time=2.0, x=-5.0)
func2d_t2.plot.surface()

In [ ]:
func2d_t2.plot.contour()

lets use some of `xarrays` powerful indexing to pull out a point at an index.

In [ ]:
function_on_grid.sel(x=0.0, y=1.0, z=3.0, time=3.0, method='nearest')

## Working With Pandas  
https://docs.xarray.dev/en/v2023.06.0/user-guide/pandas.html

## Conclusion

Hopefully this basic intro has given you the tools to work with data in `DataArrays` with using `xarray`, and given you a sneak peak into how it might be useful. Next up we are going to cover collections of data in `Datasets`.
